In [ ]:
#bkup用　ひとまず動いてもっともらしかったらどんどんここに追記

各事例ごとの水蒸気フラックスの平均場を描画する

In [ ]:
#csvファイルを読み込む　100-80 ratio2.5
#まず水蒸気フラックスの平均場
#ひとまず動作テスト　後で海陸や2020年以降も一緒にファイルに保存
csvfile = '/mnt/jet12/makoto/extract_senjo/ext_sun/csv/100-80/total_4-10_2000-2020_ratio2.5_onlytohok.csv'

#計算に必要な情報の抜出　開始時間、終了時間、ID、継続時間
#df = pd.read_csv(csvdir, dtype={'dtst': str, 'dten': str, 'hrid': str})
df = pd.read_csv(csvfile, usecols=["hrid", "dtst", "dten","nt"],dtype={"dtst":str, "dten":str, "nt":int})
for i in range(0,18):
    row = df.iloc[i]
    hrid = row["hrid"] #ID
    dtst = row["dtst"] #開始時間
    dten = row["dten"] #終了時間
    nt  =  row["nt"]   #継続時間

    #データ読み込み
    #各事例で対象期間中のデータを順次openし結果を保存
    #まず初めの時間を処理

    

    #計算用にintに変換
#    hh1 = int(str(dtst)[-2:])
    hh = int(str(dtst)[-2:])
    hh = hh - 1 #ループの都合上-1
    dd = int(str(dtst)[6:8])
    mm = int(str(dtst)[4:6])
    yyyy = int(str(dtst)[:4])
    print(dd, type(dd))
    print(hh, type(hh))

    nt = int(row["nt"])
    print(nt, type(nt))

    #初期化
    array = np.zeros((577, 721))

    #ntの時間内（１事例ずつ）ループ
    for i in range(nt+1):

        hh = hh + 1
        YYYY, MM, DD, HH = add_hours(yyyy,mm,dd,hh)

       #ゼロ埋め
        MM = f"{MM:02d}"
        DD = f"{DD:02d}"
        HH = f"{HH:02d}"

        print(YYYY)
        print(MM)
        print(DD)
        print(HH)
         
#        YYYY1 = int(dtst[:4])
#        YM =int(dtst[:6])
        #2021年7月4日12:00以降は格納ディレクトリが異なるので注意
        if YYYY < 2021:
             datadir = f'/mnt/hail1/regional_RA/GRIB2/prs0/{YYYY}/ctrl/fcst_prs_{YYYY}{MM}{DD}{HH}00.grib2'
    
        else:
             datadir = f'/mnt/hail1/regional_RA/GRIB2/prs/{YYYY}/ctrl/fcst_prs_{YYYY}{MM}{DD}{HH}00.grib2'

        print(datadir)
        grb_file_xr = xr.open_dataset(datadir, engine='cfgrib')
        Ds = grb_file_xr#.isel(isobaricInhPa=2)
        grb_file_xr.close()

        hh = int(hh)

        dten_1 = f"{YYYY}{MM}{DD}{HH}"
        print(dten_1)
        
        #環境場の計算　一つの事例に対して
        #水蒸気フラックス
        #YYYY MM DD HH が dtenと一致するまで積算を繰り返す
         
        qV=moisture_flux(Ds)
        array += qV        

        if  dten_1 == dten:
            qV=moisture_flux(Ds) #事例の継続時間の最後になったらこれまでの積算を描画
            array += qV

            ave_qV = array / nt #平均

            #データ描画

            #描画
#            ave_qV = array[:].reshape(577,721)

            lat = Ds['latitude'] ; lon = Ds['longitude']#経度緯度
            fig = plt.figure(figsize=(12,8),facecolor="white")
            plt.rcParams["figure.dpi"] = 400            # dpi(dots per inch)
            ax = fig.add_subplot(111,projection=ccrs.PlateCarree())
            #水蒸気フラックス
            im = ax.contourf(lon,lat,ave_qV,transform=ccrs.PlateCarree(),cmap='YlGnBu',interpolation='nearest',extend='both')#,cmap='turbo'
            cbar = fig.colorbar(im,ax=ax,orientation='horizontal',pad=0.05,shrink=0.62,extendfrac='auto',extendrect='True',
                                    drawedges=True,label='[g m$^{-2}$ s$^{-1}$]')#######################################
    
            countries_10m = cfeature.NaturalEarthFeature('cultural', 'admin_1_states_provinces_lines','10m', edgecolor='black', facecolor='none')    
    
            #東北地方周辺
            lat_d, lat_u = 36.0, 44.0
            lon_l, lon_r = 136.0, 145.0
            ax.set_extent([lon_l,lon_r,lat_d,lat_u])
            ax.coastlines(resolution='10m',lw=0.5) #海 岸 線 を 追 加
            ax.add_feature(countries_10m, linestyle='dotted', linewidth=0.5, alpha=1.0) #県 境 を 追 加
            gl = ax.gridlines(ccrs.PlateCarree(), draw_labels=True, linewidth=1, linestyle=':')      # 緯度線、経度線を描く
            gl.xlocator = mticker.FixedLocator(np.arange(-180, 180, 1))  # 経度線の設定
            gl.ylocator = mticker.FixedLocator(np.arange(-90, 90, 1))    # 緯度線の設定
            gl.top_labels = False
            gl.right_labels = False
            ax.set_title(f'水蒸気フラックス @950[hPa] {dtst}-{dten}',fontsize=12)###################

            plt.show()

            #配列のリセットおよびhhの処理
            array=np.zeros((577*721))
    
            hh = int(hh)

水蒸気フラックスの関数（島村さん作成）

In [ ]:
#水蒸気供給
#水蒸気フラックス計算のための関数
def moisture_flux(Ds):
    s = Ds['q']#比湿
    Wu = Ds['u']#東西風
    Wv = Ds['v']#南北風
    temp = Ds['t']#気温
    lev = Ds['isobaricInhPa']#等圧面

    #水蒸気量の計算
    dd =  lev / (2.87 * temp)#乾燥空気の密度
    # kg/kg
    q0 = s/(1-s)#混合比
     # g/kg
    q1 = q0 * 1000
    # g /m3
    q = q1 * dd#水蒸気密度

    uv = mpcalc.wind_speed(Wu[2],Wv[2]) #[2]=.isel(isobaricInhPa=2)
    qV = q[2]*uv
    qV = qV*units['g/m**3']#水蒸気フラックス

    return  qV

YYYY MM DD HH を代入し、ループで加算する際に正しい日付に修正する関数（自作）

In [ ]:
#時間繰り、日にち繰りの関数
#forはあくまで外でやる　YYYY MM DDをチェックして変換
def add_hours(YYYY, MM,  DD,  HH):

    #YYYY MM DD HHに分解
#    YYYY = int(str(dtst)[:4])  
#    MM   = int(str(dtst)[4:6])
#    DD   = int(str(dtst)[6:8])
#    HH   = int(str(dtst)[8:10])

    #月末を指定
    month_end = {
        1:31, 2:28, 3: 31, 4:30, 5:31, 6:30, 7:31, 8:31, 9:30, 10:31, 11:30, 12:31}

#    HH += 1
    if HH == 24: #足し算によって24を超えてしまう場合
        HH = 0
        DD += 1

    elif HH > 24:
        HH = HH - 24
        DD += 1

    if DD > month_end[MM]:
        DD = 1
        MM += 1

    if MM > 12:
        MM = 1
        YYYY +1 == 1

    return YYYY, MM, DD, HH